# headline

Yahoo Finance API documentation: [here](http://theautomatic.net/yahoo_fin-documentation)

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yahoo_fin.stock_info as si
from datetime import date
from datetime import datetime
import time
import sched
import csv
from streamz import Stream
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

In [108]:
tickers = [
  'AMD', 'INTC', 'NVDA', 'MSFT', 'TSLA',
  'SOW.DE', '7CD.F', 'ATVI', 'TTWO', 'CON.DE',
  'FRA.DE', 'FOOD.TO'
]

In [109]:
def get_livedata(ticker):
  livedata = {}
  liveprice = si.get_live_price(ticker)
  now = datetime.now()
  livedata['TICKER'] = ticker
  livedata['PRICE'] = liveprice
  livedata['DATE'] = f'{now.month}/{now.day}'
  livedata['TIME'] = f'{now.hour}:{now.minute}:{now.second}'
  return(livedata)

In [110]:
filename = 'livedata.csv'

def csv_init(header):
  with open(filename, 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(header)

def csv_write(livedata):
  row = list(livedata.values())
  with open(filename, 'a') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(row)

def scrape_livedata(ticker, source):
  livedata = get_livedata(ticker)
  source.emit(livedata)

In [122]:
#csv_init(columns)
source = Stream()
source.sink(csv_write)

scheduler = BackgroundScheduler()
now = datetime.now()
start = datetime(now.year, now.month, now.day, 8)
end = datetime(now.year, now.month, now.day, 22)

for i, ticker in enumerate(tickers):
  delay = (2 * i) % 59 
  scheduler.add_job(
    scrape_livedata,
    'cron',
    day_of_week='0-4',
    hour='16-21',
    minute='*',
    second=f'{0+delay}',
    kwargs={'ticker': ticker, 'source': source})

scheduler.start()

In [123]:
scheduler.shutdown(wait=False)

# Tools

In [28]:
# widgets
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, Markdown, clear_output
# finance
import yahoo_fin.stock_info as si

button = widgets.Button(description='Run!')
input_ticker = widgets.Text(value='NVDA')
out = widgets.Output()

def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          ticker = input_ticker.value
          liveprice = si.get_live_price(ticker)
          tfin = si.get_financials(ticker)
          yearly_balance_sheet = tfin['yearly_balance_sheet']
          goodwill = yearly_balance_sheet.iloc[:,0]['goodWill']
          print(f'Live Price: {liveprice}')
          print(f'Goodwill: {goodwill}')
# linking button and function together using a button's method
button.on_click(on_button_clicked)

# displaying button and its output together

row1 = HBox([input_ticker, button])
row2 = HBox([out, ])
stack = VBox([row1, row2])
stack

In [ ]:
ticker = 'NVDA'

# Tests

In [27]:
ticker = 'NVDA'
tfin = si.get_financials(ticker)
yearly_income_statement = tfin['yearly_income_statement']
yearly_balance_sheet = tfin['yearly_balance_sheet']
yearly_cash_flow = tfin['yearly_cash_flow']
quarterly_income_statement = tfin['quarterly_income_statement']
quarterly_balance_sheet = tfin['quarterly_balance_sheet']
quarterly_cash_flow = tfin['quarterly_cash_flow']
yearly_balance_sheet.iloc[:,0]['goodWill']

618000000.0

In [15]:
si.get_analysts_info(ticker)

{'Earnings Estimate':   Earnings Estimate  Current Qtr. (Jul 2020)  Next Qtr. (Oct 2020)  \
 0   No. of Analysts                    30.00                 30.00   
 1     Avg. Estimate                     1.97                  2.16   
 2      Low Estimate                     1.93                  1.66   
 3     High Estimate                     2.17                  2.69   
 4      Year Ago EPS                     1.24                  1.78   
 
    Current Year (2021)  Next Year (2022)  
 0                35.00             35.00  
 1                 8.12              9.88  
 2                 6.99              8.22  
 3                 9.31             11.98  
 4                 5.79              8.12  ,
 'Revenue Estimate':           Revenue Estimate Current Qtr. (Jul 2020) Next Qtr. (Oct 2020)  \
 0          No. of Analysts                      31                   31   
 1            Avg. Estimate                   3.65B                3.95B   
 2             Low Estimate           

In [16]:
si.get_financials(ticker)

{'yearly_income_statement': endDate                             2020-01-26   2019-01-27  2018-01-28  \
 Breakdown                                                                 
 researchDevelopment                 2829000000   2376000000  1797000000   
 effectOfAccountingCharges                 None         None        None   
 incomeBeforeTax                     2970000000   3896000000  3196000000   
 minorityInterest                          None         None        None   
 netIncome                           2796000000   4141000000  3047000000   
 sellingGeneralAdministrative        1093000000    991000000   815000000   
 grossProfit                         6768000000   7171000000  5822000000   
 ebit                                2846000000   3804000000  3210000000   
 operatingIncome                     2846000000   3804000000  3210000000   
 otherOperatingExpenses                    None         None        None   
 interestExpense                      -52000000    -58000000 

In [10]:
df = si.get_data(stock, start_date='01/01/2018', interval='1d')
df.sort_index(ascending=False).head()

,open,high,low,close,adjclose,volume,ticker
2020-07-17,409.019989,409.940002,403.510010,408.059998,408.059998,6654100,NVDA
2020-07-16,400.600006,408.269989,395.820007,405.390015,405.390015,8624100,NVDA
2020-07-15,416.570007,417.320007,402.230011,409.089996,409.089996,10099600,NVDA
2020-07-14,403.000000,416.109985,391.420013,415.079987,415.079987,13566900,NVDA
2020-07-13,423.799988,431.690002,401.000000,402.089996,402.089996,11426900,NVDA


In [17]:
si.get_live_price(ticker)

408.05999755859375

## get lists of tickers

In [20]:
dow = si.tickers_dow()
print(dow)

['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT', 'XOM']


In [21]:
dow = si.tickers_nasdaq()
print(dow)

['AACG', 'AACQU', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPL', 'AAWW', 'AAXJ', 'AAXN', 'ABCB', 'ABEO', 'ABIO', 'ABMD', 'ABTX', 'ABUS', 'ACAD', 'ACAM', 'ACAMU', 'ACAMW', 'ACBI', 'ACCD', 'ACER', 'ACGL', 'ACGLO', 'ACGLP', 'ACHC', 'ACHV', 'ACIA', 'ACIU', 'ACIW', 'ACLS', 'ACMR', 'ACNB', 'ACOR', 'ACRS', 'ACRX', 'ACST', 'ACT', 'ACTG', 'ACWI', 'ACWX', 'ADAP', 'ADBE', 'ADES', 'ADI', 'ADIL', 'ADILW', 'ADMA', 'ADMP', 'ADMS', 'ADP', 'ADPT', 'ADRE', 'ADRO', 'ADSK', 'ADTN', 'ADTX', 'ADUS', 'ADVM', 'ADXN', 'ADXS', 'AEGN', 'AEHR', 'AEIS', 'AEMD', 'AERI', 'AESE', 'AEY', 'AEYE', 'AEZS', 'AFH', 'AFIN', 'AFINP', 'AFMD', 'AFYA', 'AGBA', 'AGBAR', 'AGBAU', 'AGBAW', 'AGEN', 'AGFS', 'AGFSW', 'AGIO', 'AGLE', 'AGMH', 'AGNC', 'AGNCM', 'AGNCN', 'AGNCO', 'AGNCP', 'AGRX', 'AGTC', 'AGYS', 'AGZD', 'AHCO', 'AHPI', 'AIA', 'AIH', 'AIHS', 'AIKI', 'AIMC', 'AIMT', 'AINV', 'AIQ', 'AIRG', 'AIRR', 'AIRT', 'AIRTP', 'AIRTW', 'AKAM', 'AKBA', 'AKCA', 'AKER', 'AKRO', 'AKTS', 'AKTX', 'AKUS', 'ALAC', 'ALACR', 'ALACU', 'ALACW', 'ALBO', 'ALC

In [22]:
sp500 = si.tickers_sp500()
print(sp500)

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DL

# api börse frankfurt

## source
https://www.boerse-frankfurt.de/aktie/hellofresh-se

## historic

https://api.boerse-frankfurt.de/data/price_history?limit=50&offset=0&isin=DE0007236101&mic=XFRA&minDate=2019-04-09&maxDate=2020-04-09
https://api.boerse-frankfurt.de/data/historical_key_data?isin=DE0007236101&limit=5

## stream

daily information livestream
https://api.boerse-frankfurt.de/data/price_information?isin=DE000A161408&mic=XETR

ask/bid livestream
https://api.boerse-frankfurt.de/data/bid_ask_overview?isin=DE000A161408&mic=

quotebox?
https://api.boerse-frankfurt.de/data/quote_box?isin=DE000A161408&mic=XETR

news
https://api.boerse-frankfurt.de/data/instrument_news?isin=DE000A161408&limit=2&newsType=ALL